In [1]:
from pathlib import Path, PosixPath

import pandas as pd
from sqlalchemy import create_engine

In [2]:
def load_sql_or_csv(fname: PosixPath) -> pd.DataFrame:
    """Either load saved csv or load from database and save as csv"""
    if fname.exists():
        print('loading csv')
        return pd.read_csv(fname, dtype={'cms_certification_num': str})
    else:
        print('loading from database')
        engine = create_engine("mysql+pymysql://root@localhost/hospital_price_transparency_v3")
        remaining_wa = pd.read_sql("select * from hospitals WHERE last_edited_by_username IS NULL AND state = 'WA'", engine)
        remaining_wa.to_csv('remaining_wa.csv', index=False)
        return remaining_wa

In [3]:
remaining_wa = load_sql_or_csv(Path('remaining_wa.csv'))
len(remaining_wa)

loading csv


85

## create hospitals.csv

In [4]:
uwmc = remaining_wa[remaining_wa['cms_certification_num'] == '500008'].copy()
harborview = remaining_wa[remaining_wa['cms_certification_num'] == '500064'].copy()

uwmc['homepage_url'] = 'https://www.uwmedicine.org/'
harborview['homepage_url'] = 'https://www.uwmedicine.org/locations/harborview-medical-center'

# same overall hopsital, different locations
uwmc['chargemaster_url'] = 'https://www.uwmedicine.org/patient-resources/billing-and-insurance/charge-description-master-codes'
harborview['chargemaster_url'] = 'https://www.uwmedicine.org/patient-resources/billing-and-insurance/charge-description-master-codes'

uwmc['last_edited_by_username'] = 'joeeoj'
harborview['last_edited_by_username'] = 'joeeoj'

hospitals = pd.concat([uwmc, harborview])

hospitals

,cms_certification_num,name,address,city,state,zip5,beds,phone_number,homepage_url,chargemaster_url,last_edited_by_username
36,500008,UNIVERSITY OF WASHINGTON MEDICAL CTR,1959 NE PACIFIC ST BOX 356151,SEATTLE,WA,98195,450,2065983300,https://www.uwmedicine.org/,https://www.uwmedicine.org/patient-resources/b...,joeeoj
58,500064,HARBORVIEW MEDICAL CENTER,325 9TH AVENUE,SEATTLE,WA,98104,413,2067313000,https://www.uwmedicine.org/locations/harborvie...,https://www.uwmedicine.org/patient-resources/b...,joeeoj


In [5]:
hospitals.to_csv('hospitals.csv', index=False)